## Test connection

In [1]:
# list all the connected USB device-----
import serial.tools.list_ports
allports = serial.tools.list_ports.comports()
for port, desc, hwid in sorted(allports):
    print(f"{port}, {desc}. {hwid}")

COM1, Communications Port (COM1). ACPI\PNP0501\2
COM3, USB Serial Port (COM3). USB VID:PID=0403:6001 SER=VDIS200A
COM4, Intel(R) Active Management Technology - SOL (COM4). PCI\VEN_8086&DEV_43E3&SUBSYS_0A541028&REV_11\3&11583659&0&B3


In [2]:
import time
import sys
import os
path_project = "\\".join(os.getcwd().split("\\")[:-2])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)

from hardware.mw.mwsynthesizer import *
from hardware import config_custom as hdc

mwsyn = Synthesizer(hdc.VDISYN_SN, 
                    vidpid=hdc.VDISYN_VIDPID,
                    baudrate=hdc.VDISYN_BAUD)


VDI Sythesizer Serail Port Open


## Test CW Frequency

In [3]:
freq = 8.33333333333
freq = 9.00000002
freq = 8.9999999999
freq = 16.6
# use lower-level functions to send cw frequency command
bytescommand = mwsyn._cw_frequency_command(freq)
print_bytestring(bytescommand)
mwsyn.serialcom.write(bytescommand)
received = mwsyn.serialcom.read(size=6)
print_bytestring(received)

# send cw frequency command
errorbyte, freq_actual = mwsyn.cw_frequency(freq)
print(f"CW Freqeuncy Setting Sent:{freq} GHz")
print(f"Actual Output CW Freqeuncy :{freq_actual} GHz")

0x06 0x46 0x10 0x99 0x99 0x99 0x9A 0x53 
0x55 0x10 0x99 0x99 0x99 0xB0 
CW Freqeuncy Setting Sent:16.6 GHz
Actual Output CW Freqeuncy :16.600000005215406 GHz


## Test Sweep
The rise/fall step time should be multiples of 4ns. Please check the minimum step size of the synthesizer to avoid frequency spurs

Here we test the example in the programmer's manual---------------

In [4]:

freq_start = 8
freq_stop = 20
step_rise = 1E-6 
step_fall = 10E-6 
steptime_rise = 1.99E3 
steptime_fall = 8.0
dwellatlow = False
dwellathigh = False
bytescommand = mwsyn._simple_sweep_command(freq_start, freq_stop, step_rise, step_fall, steptime_rise, steptime_fall, dwellatlow, dwellathigh)
print_bytestring(bytescommand)
mwsyn.serialcom.write(bytescommand)
received = mwsyn.serialcom.read(size=21)
print_bytestring(received)

0x1C 0x73 0xFF 0x08 0x00 0x00 0x00 0x00 0x14 0x00 0x00 0x00 0x00 0x00 0x00 0x00 0x10 0xC7 0x00 0x00 0x00 0xA7 0xC6 0x01 0xF2 0x00 0x02 0x00 0x00 0xCB 
0x55 0x07 0xFF 0xFF 0xFF 0xEC 0x13 0xFF 0xFF 0xFF 0xE8 0x00 0x00 0x00 0x10 0xDC 0x00 0x00 0x00 0xA7 0xC8 


Here we test with the frequencies that work with the 400GHz 24xAMC-----------

In [5]:
freq_start = 16.45-0.25
# freq_start = 16.2
freq_stop = 16.45+0.25

# check the minium step size
freqminstep_list = mwsyn.get_min_step_size([freq_start], [freq_stop])
step_min = freqminstep_list[0] # in Hz
freq_step = 0.5E-3
freq_step = int(freq_step/step_min*1E9)*step_min/1E9
print(f"Synthesizer Frequency Min. Step:{step_min}Hz")
print(f"Frequency Sweep Step:{freq_step}Hz")
step_rise = freq_step
step_fall = freq_step
# the rise/fall must be a multiple of 4ns
steptime = 262.14E3
steptime = int(steptime/4)*4
steptime_rise = steptime 
steptime_fall = steptime
# no dwell at low or high limits so it is sweeping 
# from low to high and go back low so on and so forth
dwellatlow = False
dwellathigh = False

Synthesizer Frequency Min. Step:15.832483768463135Hz
Frequency Sweep Step:0.0004999898374080658Hz


In [6]:
# lower-level functions-----
print(f"Approximated Time to Sweep along rise direction: \n{(freq_stop-freq_start)/step_rise*steptime_rise/1E9}s")
bytescommand = mwsyn._sweep_command(freq_start, freq_stop, step_rise, step_fall, steptime_rise, steptime_fall, dwellatlow, dwellathigh)
print_bytestring(bytescommand)
mwsyn.serialcom.write(bytescommand)
received = mwsyn.serialcom.read(size=21)
print_bytestring(received)
mwsyn.reset_trigger()
mwsyn.sweep_continue()
time.sleep(5) #sweep for 5s
mwsyn.sweep_pause()

Approximated Time to Sweep along rise direction: 
0.2621453281519949s
0x1C 0x53 0xFF 0x10 0x33 0x33 0x33 0x33 0x10 0xB3 0x33 0x33 0x33 0x00 0x00 0x20 0xC4 0x70 0x00 0x00 0x20 0xC4 0x70 0xFF 0xFF 0xFF 0xFF 0x00 0x00 0x30 
0x55 0x10 0x33 0x33 0x33 0x34 0x10 0xB3 0x33 0x33 0x3C 0x00 0x00 0x20 0xC4 0x70 0x00 0x00 0x20 0xC4 0x70 


b'\xab'

In [7]:
# higher-level function-----
freq_start_actual, freq_stop_actual, step_rise_actual, step_fall_actual = mwsyn.sweep(freq_start, freq_stop, step_rise, step_fall, 
                                                                                      steptime_rise, steptime_fall, dwellatlow, dwellathigh)
print(f"""
Acutal Sweep Parameters:
        Freq Start:{freq_start_actual}
        Freq Stop:{freq_stop_actual}
        Rise Step:{step_rise_actual}
        Fall Step:{step_fall_actual}""")
mwsyn.reset_trigger()
mwsyn.sweep_continue()
time.sleep(5) #sweep for 5s
mwsyn.sweep_pause()


Acutal Sweep Parameters:
        Freq Start:16.200000000186265
        Freq Stop:16.70000000204891
        Rise Step:0.0004999898374080658
        Fall Step:0.0004999898374080658


b'\xab'

# Other tests

In [4]:
mwsyn.reboot()

MW Synthesizer Rebooted!


In [5]:
mwsyn.open()

In [6]:
mwsyn.close()